In [ ]:
!pip install sevenn
!pip install git+https://github.com/MDIL-SNU/SevenNet.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.0/448.0 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.8/448.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
import os
# Set the environment variable to reduce fragmentation before any GPU memory is allocated
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from google.colab import drive
drive.mount('/content/drive')

# Import the calculator and trigger the checkpoint download.
from sevenn.calculator import SevenNetCalculator

# Instantiate a dummy calculator to ensure the checkpoint is downloaded.
dummy_calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')





print("Checkpoint downloaded successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Checkpoint downloaded successfully!


In [ ]:
import os
import csv
import gc
import torch

from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from sevenn.calculator import SevenNetCalculator

base_input_dir = '/content/drive/MyDrive/ALL'
calc = dummy_calc

skip_systems = {
    "Name of the folder, if any systems to skip",
}

for root, dirs, files in os.walk(base_input_dir):
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: POSCAR files not found.")
            continue

        system_folder = root
        system_name = os.path.basename(system_folder)
        save_dir = os.path.join(system_folder, "SevenNet_run_ACTUAL")
        csv_path = os.path.join(save_dir, "energies.csv")

        if system_name in skip_systems or os.path.exists(csv_path):
            reason = "manual skip list" if system_name in skip_systems else "energies.csv exists"
            print(f"Skipping system {system_name} as {reason}.")
            skip_systems.add(system_name)
            continue

        print(f"Processing system: {system_name}")
        os.makedirs(save_dir, exist_ok=True)

        try:
            initial = read(initial_path)
            final = read(final_path)

            mask_initial = [atom.tag > 1 for atom in initial]
            initial.set_constraint(FixAtoms(mask=mask_initial))
            mask_final = [atom.tag > 1 for atom in final]
            final.set_constraint(FixAtoms(mask=mask_final))

            initial.calc = calc
            final.calc = calc

            init_traj = os.path.join(save_dir, "initial_relaxed.traj")
            final_traj = os.path.join(save_dir, "final_relaxed.traj")

            print(f"  ➤ Relaxing initial...")
            qn_init = QuasiNewton(initial, trajectory=init_traj)
            qn_init.run(fmax=0.05, steps=1000)
            write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

            torch.cuda.empty_cache()
            gc.collect()

            print(f"  ➤ Relaxing final...")
            qn_final = QuasiNewton(final, trajectory=final_traj)
            qn_final.run(fmax=0.05, steps=1000)
            write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

            torch.cuda.empty_cache()
            gc.collect()

            initial = read(init_traj)
            final = read(final_traj)
            constraint = FixAtoms(mask=[atom.tag > 1 for atom in final])

            images = [initial]
            for i in range(3):
                image = initial.copy()
                image.set_constraint(constraint)
                image.calc = calc
                images.append(image)
            images.append(final)

            print(f"  ➤ Running NEB...")
            neb = NEB(images, parallel=False, k=5, method='eb', allow_shared_calculator=True)
            neb.interpolate('idpp')
            neb_traj = os.path.join(save_dir, "neb.traj")

            qn_neb = BFGS(neb, trajectory=neb_traj)
            qn_neb.run(fmax=0.05, steps=1000)

            torch.cuda.empty_cache()
            gc.collect()

            energies = []
            with open(csv_path, mode="w", newline="") as file:
                writer = csv.writer(file)
                writer.writerow(["Image Index", "Energy (eV)"])
                for i, image in enumerate(images):
                    try:
                        energy = image.get_potential_energy()
                    except Exception as e:
                        print(f"Error computing energy for image {i} of {system_name}: {e}")
                        energy = None
                    energies.append(energy)
                    output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                    write(output_file, image, format="vasp")
                    writer.writerow([i, energy])

            if None in energies:
                print(f"Skipping migration barrier for {system_name} due to energy errors.")
                continue

            migration_barrier = max(energies) - min(energies)
            print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
            with open(csv_path, mode="a", newline="") as file:
                writer = csv.writer(file)
                writer.writerow([])
                writer.writerow(["Migration Barrier", migration_barrier])

        except Exception as e:
            print(f"Error in {system_name}: {e}")
            torch.cuda.empty_cache()
            gc.collect()
            continue

print("\n All NEB calculations complete!")

# Print skipped systems
print("\nSkipped systems:")
for name in sorted(skip_systems):
    print(f" - {name}")



Streaming output truncated to the last 5000 lines.
  ➤ Relaxing initial...
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 07:38:17     -449.769806       2.1027
BFGSLineSearch:    1[  2] 07:38:18     -450.662048       0.2093
BFGSLineSearch:    2[  4] 07:38:18     -450.685425       0.1091
BFGSLineSearch:    3[  5] 07:38:18     -450.690216       0.0833
BFGSLineSearch:    4[  6] 07:38:18     -450.691162       0.0461
  ➤ Relaxing final...
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 07:38:19     -449.769806       2.1027
BFGSLineSearch:    1[  2] 07:38:19     -450.662048       0.2093
BFGSLineSearch:    2[  4] 07:38:19     -450.685455       0.1091
BFGSLineSearch:    3[  5] 07:38:19     -450.690216       0.0833
BFGSLineSearch:    4[  6] 07:38:20     -450.691162       0.0461
  ➤ Running NEB...
      Step     Time          Energy          fmax
BFGS:    0 07:38:21     -449.886841        1.194672
BFGS:    1 0

In [ ]:
import csv
import os
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from sevenn.calculator import SevenNetCalculator
import torch

# Disable debugging validation to address frozen modules warning
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

# Check available GPUs and set device
print("Number of GPUs:", torch.cuda.device_count())
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Base input directory
base_input_dir = '/content/drive/MyDrive/MLIP_NEB_inputs'

# Shared calculator for endpoints
shared_calc = dummy_calc

# Walk through the directory tree
for root, dirs, files in os.walk(base_input_dir):
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: initial or final POSCAR not found.")
            continue

        system_name = os.path.basename(root)
        print(f"Processing system: {system_name}")

        save_dir = os.path.join(root, "SevenNet_run")
        os.makedirs(save_dir, exist_ok=True)
        csv_path = os.path.join(save_dir, "energies.csv")

        # Read and constrain structures
        initial = read(initial_path)
        final = read(final_path)
        mask_initial = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask_initial))
        mask_final = [atom.tag > 1 for atom in final]
        final.set_constraint(FixAtoms(mask=mask_final))

        # Assign shared calculator
        initial.calc = shared_calc
        final.calc = shared_calc

        # Relax initial structure
        init_traj = os.path.join(save_dir, "initial_relaxed.traj")
        qn = QuasiNewton(initial, trajectory=init_traj)
        qn.run(fmax=0.05, steps=500)
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

        # Relax final structure
        final_traj = os.path.join(save_dir, "final_relaxed.traj")
        qn = QuasiNewton(final, trajectory=final_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Reload relaxed structures
        initial = read(init_traj)
        final = read(final_traj)
        mask_final = [atom.tag > 1 for atom in final]
        constraint = FixAtoms(mask=mask_final)

        # Create NEB images with fewer intermediates to reduce memory usage
        images = [initial]
        num_intermediates = 3  # Reduced from 7 to 3
        for i in range(num_intermediates):
            image = initial.copy()
            # Use a new calculator per image, but we'll mitigate memory later
            image.calc = SevenNetCalculator('7net-mf-ompa', modal='omat24', device=device)
            image.set_constraint(constraint)
            images.append(image)
        images.append(final)

        # Clear GPU memory before NEB
        torch.cuda.empty_cache()
        print(f"GPU memory allocated before NEB: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
        print(f"GPU memory reserved before NEB: {torch.cuda.memory_reserved()/1024**3:.2f} GB")

        # Run NEB with parallel=False to avoid GPU memory overload
        try:
            neb = NEB(images, parallel=False, k=5, method='eb')  # Changed to parallel=False
            neb.interpolate('idpp')
            neb_traj = os.path.join(save_dir, "neb.traj")
            qn = BFGS(neb, trajectory=neb_traj)
            qn.run(fmax=0.05, steps=1000)
        except Exception as e:
            print(f"NEB failed for {system_name}: {e}. Skipping.")
            torch.cuda.empty_cache()
            continue

        # Compute and save energies
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Energy error for image {i} of {system_name}: {e}")
                    energy = None
                energies.append(energy)
                output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                write(output_file, image, format="vasp")
                writer.writerow([i, energy])

        if None in energies:
            print(f"Skipping barrier calculation for {system_name} due to energy errors")
            torch.cuda.empty_cache()
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

        torch.cuda.empty_cache()

print("✅ All NEB Calculations complete!")

Number of GPUs: 1
Using device: cuda
Processing system: P2Si
                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 06:45:36     -277.955536       0.6020
BFGSLineSearch:    1[  2] 06:45:38     -278.014648       0.2591
BFGSLineSearch:    2[  4] 06:45:39     -278.046814       0.4859
BFGSLineSearch:    3[  6] 06:45:39     -278.061188       0.1513
BFGSLineSearch:    4[  8] 06:45:40     -278.064941       0.1375
BFGSLineSearch:    5[ 10] 06:45:41     -278.067841       0.0945
BFGSLineSearch:    6[ 12] 06:45:41     -278.070557       0.1222
BFGSLineSearch:    7[ 14] 06:45:41     -278.072571       0.0950
BFGSLineSearch:    8[ 16] 06:45:42     -278.074310       0.0705
BFGSLineSearch:    9[ 18] 06:45:42     -278.075623       0.0577
BFGSLineSearch:   10[ 20] 06:45:42     -278.076477       0.0626
BFGSLineSearch:   11[ 22] 06:45:42     -278.077362       0.0831
BFGSLineSearch:   12[ 24] 06:45:43     -278.077972       0.0538
BFGSLineSearch:   13[ 25] 06:45:43     -27

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Import the calculator and trigger the checkpoint download.
from sevenn.calculator import SevenNetCalculator

# Instantiate a dummy calculator. This will check for and download the checkpoint if it isn't already present.
dummy_calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')
del dummy_calc

print("Checkpoint downloaded successfully!")


Mounted at /content/drive


Checkpoint downloaded: /usr/local/lib/python3.11/dist-packages/sevenn/pretrained_potentials/SevenNet_MF_ompa/checkpoint_sevennet_mf_ompa.pth


In [ ]:
import os
import csv
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from sevenn.calculator import SevenNetCalculator

# Set device to GPU (A100) if needed, e.g., device = 'cuda'
base_input_dir = '/content/drive/MyDrive/MLIP_NEB_inputs'

# Instantiate a shared calculator instance. The checkpoint is already available from Cell 1.
calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')

# Walk through the directory tree under MLIP_NEB_inputs
for root, dirs, files in os.walk(base_input_dir):
    # Check if both "initial" and "final" folders are present in the current directory
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        # Only process if both POSCAR files exist
        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: initial or final POSCAR not found.")
            continue

        system_folder = root  # This is the system's folder
        system_name = os.path.basename(system_folder)
        print(f"Processing system: {system_name}")

        # Create a subfolder in the system folder for the NEB outputs
        save_dir = os.path.join(system_folder, "SevenNet_run")
        os.makedirs(save_dir, exist_ok=True)

        # CSV file to record energies
        csv_path = os.path.join(save_dir, "energies.csv")

        # Read initial and final structures
        initial = read(initial_path)
        final = read(final_path)

        # Apply constraints: fix atoms with tag > 1
        mask_initial = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask_initial))
        mask_final = [atom.tag > 1 for atom in final]
        final.set_constraint(FixAtoms(mask=mask_final))
        # We'll use the final mask for the NEB images

        # Assign the shared calculator to both endpoints
        initial.calc = calc
        final.calc = calc

        # Relax the initial structure (steps reduced for testing)
        init_traj = os.path.join(save_dir, "initial_relaxed.traj")
        qn = QuasiNewton(initial, trajectory=init_traj)
        qn.run(fmax=0.05, steps=500)
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

        # Relax the final structure (steps reduced for testing)
        final_traj = os.path.join(save_dir, "final_relaxed.traj")
        qn = QuasiNewton(final, trajectory=final_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Reload relaxed structures
        initial = read(init_traj)
        final = read(final_traj)
        mask_final = [atom.tag > 1 for atom in final]  # Recalculate based on relaxed final
        constraint = FixAtoms(mask=mask_final)

        images = [initial]
        for i in range(7):
            image = initial.copy()
            # Assign a fresh SevenNet calculator to each image
            image.calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')
            image.set_constraint(constraint)
            images.append(image)
        images.append(final)

        # Set up and run the NEB calculation with error handling for memory issues
        try:
            neb = NEB(images, parallel=True, k=5, method='eb')
            neb.interpolate('idpp')
            neb_traj = os.path.join(save_dir, "neb.traj")
            qn = BFGS(neb, trajectory=neb_traj)
            qn.run(fmax=0.05, steps=1000)
        except Exception as e:
            print(f"NEB calculation failed for system {system_name}: {e}. Skipping to the next system.")
            continue

        # Compute and save energies for each image
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {system_name}: {e}")
                    energy = None
                energies.append(energy)
                output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                write(output_file, image, format="vasp")
                writer.writerow([i, energy])

        # Check for any energy errors before calculating migration barrier
        if None in energies:
            print(f"Skipping migration barrier calculation for {system_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("✅ All NEB Calculations complete!")



In [ ]:
# Mount Google Drive in Colab with forced remount to avoid duplicate mount warnings
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import csv
from ase.constraints import FixAtoms
from ase.io import read, write
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from sevenn.calculator import SevenNetCalculator

# Set device to GPU (A100)
#device = 'cuda'  # or 'cuda:0' if preferred
base_input_dir = '/content/drive/MyDrive/MLIP_NEB_inputs'

# Instantiate a single calculator instance to be shared by all images
calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')

# Walk through the directory tree under MLIP_NEB_inputs
for root, dirs, files in os.walk(base_input_dir):
    # Check if both "initial" and "final" folders are present in the current directory
    if 'initial' in dirs and 'final' in dirs:
        initial_path = os.path.join(root, "initial", "POSCAR")
        final_path = os.path.join(root, "final", "POSCAR")

        # Only process if both POSCAR files exist
        if not (os.path.exists(initial_path) and os.path.exists(final_path)):
            print(f"Skipping {root}: initial or final POSCAR not found.")
            continue

        system_folder = root  # This is the system's folder
        system_name = os.path.basename(system_folder)
        print(f"Processing system: {system_name}")

        # Create a subfolder in the system folder for the NEB outputs
        save_dir = os.path.join(system_folder, "SevenNet_run")
        os.makedirs(save_dir, exist_ok=True)

        # CSV file to record energies
        csv_path = os.path.join(save_dir, "energies.csv")
        if os.path.exists(csv_path):
          print(f"Skipping {structure_name}: energies.csv already exists.")
          continue

        # Read initial and final structures
        initial = read(initial_path)
        final = read(final_path)

        # Apply constraints: fix atoms with tag > 1
        mask_initial = [atom.tag > 1 for atom in initial]
        initial.set_constraint(FixAtoms(mask=mask_initial))
        mask_final = [atom.tag > 1 for atom in final]
        final.set_constraint(FixAtoms(mask=mask_final))
        # We'll use the final mask for the NEB images

        # Assign the shared calculator to both endpoints

        initial.calc = calc
        final.calc = calc

        # Relax the initial structure (steps reduced for testing)
        init_traj = os.path.join(save_dir, "initial_relaxed.traj")
        qn = QuasiNewton(initial, trajectory=init_traj)
        qn.run(fmax=0.05, steps=500)
        write(os.path.join(save_dir, "relaxed_initial.vasp"), initial, format="vasp")

        # Relax the final structure (steps reduced for testing)
        final_traj = os.path.join(save_dir, "final_relaxed.traj")
        qn = QuasiNewton(final, trajectory=final_traj)
        qn.run(fmax=0.05, steps=1000)
        write(os.path.join(save_dir, "relaxed_final.vasp"), final, format="vasp")

        # Reload relaxed structures
        initial = read(init_traj)
        final = read(final_traj)

        # Create NEB images (7 intermediate images)
        constraint = FixAtoms(mask=mask_final)
        images = [initial]
        for i in range(7):
            image = initial.copy()
            # Assign a fresh CHGNet calculator to each image
            image.calc = SevenNetCalculator('7net-mf-ompa', modal='omat24')
            image.set_constraint(constraint)
            images.append(image)
        images.append(final)

        # Set up and run the NEB calculation with parallel evaluation turned off to reduce resource usage
        neb = NEB(images, parallel=True, k=5, method='eb')
        neb.interpolate('idpp')
        neb_traj = os.path.join(save_dir, "neb.traj")
        qn = BFGS(neb, trajectory=neb_traj)
        qn.run(fmax=0.05, steps=1000)

        # Compute and save energies for each image
        energies = []
        with open(csv_path, mode="w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["Image Index", "Energy (eV)"])
            for i, image in enumerate(images):
                try:
                    energy = image.get_potential_energy()
                except Exception as e:
                    print(f"Error computing energy for image {i} of {system_name}: {e}")
                    energy = None
                energies.append(energy)
                output_file = os.path.join(save_dir, f"neb_optimized_{i}.vasp")
                write(output_file, image, format="vasp")
                writer.writerow([i, energy])

        # Check for any energy errors before calculating migration barrier
        if None in energies:
            print(f"Skipping migration barrier calculation for {system_name} due to energy errors")
            continue

        migration_barrier = max(energies) - min(energies)
        print(f"{system_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_path, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

print("✅ All NEB Calculations complete!")


In [ ]:
"""
Collect migration-barrier values from every SevenNet_run_ACTUAL folder and
write them to a CSV whose location you choose.

✓ Works in Google Colab (after drive.mount) or on your local machine.
"""

import csv
import pathlib
import sys
from google.colab import drive
drive.mount('/content/drive')
# then run the script cell above, changing DEST_CSV if you like
# ---------------------------------------------------------------------------
# 1) --- CONFIGURATION -------------------------------------------------------
# ---------------------------------------------------------------------------
ROOT = pathlib.Path("/content/drive/MyDrive/ALL")   # top directory to scan
TARGET_DIRNAME = "ORBV2_run"              # folder to locate
ENERGY_FILE = "energies.csv"                        # file inside that folder

# --- Choose the destination for the summary CSV ----------------------------
# Option A – hard-code it here:
DEST_CSV = pathlib.Path("/content/drive/MyDrive/Results/OrbV2.csv")

# Option B – or supply it on the command line:
if len(sys.argv) == 2:
    DEST_CSV = pathlib.Path(sys.argv[1])

# ---------------------------------------------------------------------------
# 2) --- MAIN ----------------------------------------------------------------
# ---------------------------------------------------------------------------
results = []

for snet_dir in ROOT.rglob(TARGET_DIRNAME):
    if not snet_dir.is_dir():
        continue

    parent_name = snet_dir.parent.name
    energies_path = snet_dir / ENERGY_FILE

    if not energies_path.exists():
        print(f"⚠️  Skip {snet_dir}: {ENERGY_FILE} not found")
        continue

    # read last non-blank line
    last_line = next((line.strip() for line in reversed(energies_path.read_text().splitlines()) if line.strip()), None)
    if last_line is None:
        print(f"⚠️  Skip {snet_dir}: {ENERGY_FILE} empty")
        continue

    # split on comma or whitespace; take final token
    token = last_line.replace(",", " ").split()[-1]
    try:
        barrier = float(token)
    except ValueError:
        print(f"⚠️  Skip {snet_dir}: cannot parse number in → {last_line!r}")
        continue

    results.append((parent_name, barrier))
    print(f"✔︎  {parent_name:25s}  {barrier}")

# ---------------------------------------------------------------------------
# 3) --- WRITE THE OUTPUT CSV ------------------------------------------------
# ---------------------------------------------------------------------------
if not results:
    print("\nNo valid records found – nothing to write.")
    sys.exit(0)

# make sure the parent folder exists
DEST_CSV.parent.mkdir(parents=True, exist_ok=True)

with DEST_CSV.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["parent_directory", "migration_barrier"])
    writer.writerows(sorted(results))

print(f"\nSaved {len(results)} entries to: {DEST_CSV}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✔︎  Ca3Ga2N4                   2.1689453125
✔︎  g-Li3PO4_1                 0.56561279296875
✔︎  e-VOPO4_Na_1               1.3241729736328125
✔︎  Ti2F7_Ca_2                 0.77294921875
✔︎  MgMnSiS4                   0.720947265625
✔︎  O-LiCoPO4                  0.23028564453125
✔︎  Ca3Cr2Si3O12               2.33251953125
✔︎  MnPO4F_Mg                  0.5592041015625
✔︎  Cr2S4_Mg                   0.540130615234375
✔︎  b-Li3PS4_4                 0.41656494140625
✔︎  CaCu2S2_1                  1.781341552734375
✔︎  CaMn2O4_P1                 0.389404296875
✔︎  MgDy2S4                    0.511871337890625
✔︎  Na2NiSiO4_1                3.79638671875
✔︎  MgNd2S4                    0.564056396484375
✔︎  CaNaV2P2SO12_1             0.943145751953125
✔︎  ZnMn2O4                    0.2979736328125
✔︎  Cr2F7_Ca_1                 1.375885009765625
✔︎  MgNi2O4       